## Metaprogaming in Julia
### Program representation
`String` => `Expr`
- `Meta.parse()::Expr`, `Expr` has two field
  - `.head`: identifying the kind of expression.
  - `args`: the expression arguments, which may be symbols, orther expression, or literal values
- `Expr()`

In [1]:
prog = "1+1"
ex1 = Meta.parse(prog)

:(1 + 1)

In [2]:
typeof(ex1)

Expr

In [3]:
ex1.head

:call

In [4]:
ex1.args

3-element Vector{Any}:
  :+
 1
 1

In [5]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [6]:
ex1 == ex2

true

In [7]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [8]:
ex3 = Meta.parse("(4 + 4) / 2")
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

In [9]:
dump(ex3)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol /
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 4
        3: Int64 4
    3: Int64 2


### Expressions and evaluation
#### Quoting
- `:(expr)`: create expression
- `quote ... end`: quoting for multiple expression

In [10]:
ex = :(a+b*c+1)

:(a + b * c + 1)

In [11]:
:(a + b*c + 1) == 
    Meta.parse("a + b*c + 1") ==
    Expr(:call, :+, :a, Expr(:call, :*, :b, :c),1)

true

In [12]:
ex  = quote
    x = 1
    y = 2
    x + y
end

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:2 =#
    x = 1
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:3 =#
    y = 2
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:4 =#
    x + y
end

#### Interpolation
- `$`
- `$(xs...)`: splatting interpolation

In [13]:
a = 1;
ex = :($a + b)

:(1 + b)

In [14]:
ex = :(a in $:((1,2,3)))

:(a in (1, 2, 3))

In [15]:
args = [:x,:y,:z];
:(f(1, $(args...)))

:(f(1, x, y, z))

#### Nested quote

In [16]:
x = :(1+2);
e = quote quote $x end end
    

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:2 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:2 =#
    $(Expr(:$, :x))
end))
end

In [17]:
eval(e)

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:2 =#
    1 + 2
end

In [18]:
e = quote quote $$x end end

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:$, :(1 + 2)))
end))
end

In [19]:
eval(e)

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    3
end

In [20]:
e = quote quote quote $$x end end end

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:$, :($(Expr(:$, :x)))))
end))
end))
end

In [21]:
eval(e)

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:$, :(1 + 2)))
end))
end

In [22]:
e = quote quote quote $$$x end end end

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:$, :($(Expr(:$, :(1 + 2))))))
end))
end))
end

In [23]:
eval(e)

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:1 =#
    $(Expr(:$, 3))
end))
end

#### QuoteNode

In [24]:
dump(Meta.parse(":(1+2)"))

Expr
  head: Symbol quote
  args: Array{Any}((1,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 2


In [25]:
eval(Meta.quot(Expr(:$, :(1+2))))

3

In [26]:
Meta.quot(Expr(:$, :(1+2)))

:($(Expr(:quote, :($(Expr(:$, :(1 + 2)))))))

In [27]:
x = :(1+2)
quote
    quote
        $x
    end
end

quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:3 =#
    $(Expr(:quote, quote
    #= /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb:4 =#
    $(Expr(:$, :x))
end))
end

In [28]:
dump(eval(QuoteNode(Expr(:$, :(1+2)))))

Expr
  head: Symbol $
  args: Array{Any}((1,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 2


In [29]:
dump(QuoteNode(Expr(:$, :(1+2))))

QuoteNode
  value: Expr
    head: Symbol $
    args: Array{Any}((1,))
      1: Expr
        head: Symbol call
        args: Array{Any}((3,))
          1: Symbol +
          2: Int64 1
          3: Int64 2


In [30]:
eval(Meta.parse(":x"))

:x

### Evaluating expression

In [31]:
ex1 = :(1 + 2)
eval(ex1)

3

In [32]:
ex = :(a + b)
eval(ex)

UndefVarError: UndefVarError: `b` not defined

In [33]:
a = 1; b = 2;
eval(ex)

3

In [34]:
ex = :(x = 1)
eval(ex)
x

1

In [35]:
a = 1;
ex = Expr(:call, :+, a, :b)


:(1 + b)

In [36]:
a = 1;
ex = Expr(:call, :+, a, :b)
a = 0; b = 2;
eval(ex)

3

### Functions on `Expr`

In [37]:
function math_expr(op, op1, op2)
    expr = Expr(:call, op, op1, op2)
    return expr
end

math_expr (generic function with 1 method)

In [38]:
ex = math_expr(:+, 1, Expr(:call, :*, 4,5))
eval(ex)

21

In [39]:
function make_expr2(op, opr1, opr2)
    opr1f, opr2f = map(x -> isa(x, Number) ? 2*x : x, (opr1,opr2))
    retexpr = Expr(:call, op, opr1f, opr2f)
    return retexpr
end

make_expr2 (generic function with 1 method)

In [40]:
make_expr2(:+, 1, 2)

:(2 + 4)

In [41]:
ex = make_expr2(:+, 1, Expr(:call, :*, 5, 8))

:(2 + 5 * 8)

In [42]:
eval(ex)

42

### Macros
#### Basic
- `macro ... return ::Expr end`
- `macroexpand` or `@macroexpand` return the `Expr`

In [43]:
macro sayhello()
    return :( println("Hello, world!"))
end

@sayhello (macro with 1 method)

In [44]:
@sayhello()

Hello, world!


In [45]:
macro sayhello(name)
    return :(println("Hello, " * $name))
end

@sayhello (macro with 2 methods)

In [46]:
@sayhello("Julia")

Hello, Julia


In [47]:
ex = macroexpand(Main, :(@sayhello("human")))

:(Main.println("Hello, " * "human"))

In [48]:
eval(ex)

Hello, human


In [49]:
@macroexpand @sayhello "human"
    

:(Main.println("Hello, " * "human"))

#### Why macros?
Macrrs are necessary because they execute when code is parsed, therefore, macros allow the programmer to generate and include fragments of customzied code *before* the full program is run.

In [50]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)
    return :(println("I execute at runtime. The argument is: ", $arg))
end

@twostep (macro with 1 method)

In [51]:
ex = macroexpand(Main, :(@twostep :(1,2,3)));

I execute at parse time. The argument is: :((1, 2, 3))


In [52]:
@twostep :(1,2,3)

I execute at parse time. The argument is: :((1, 2, 3))
I execute at runtime. The argument is: (1, 2, 3)


#### Macro invocation
- Special marcos
    - `__source__`
    - `__module__`
    - `@__FILE__`
    - `@__LINE__`
    - `@__DIR__`
    - `@__MODULE__`

In [53]:
macro showarg(x)
    show(x)
end

@showarg (macro with 1 method)

In [54]:
@showarg a

:a

In [55]:
@showarg 1+1

:(1 + 1)

In [56]:
@showarg(pirntln("Yo!"))

:(pirntln("Yo!"))

In [57]:
macro __LOCATION__()
    return QuoteNode(__source__) 
end

@__LOCATION__ (macro with 1 method)

In [59]:
dump(
    @__LOCATION__()
)

LineNumberNode
  line: Int64 2
  file: Symbol /Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb


In [60]:

a = @__LOCATION__
a.file

Symbol("/Users/zhengpanpan/Documents/code/Jupyter/Julia-HPC/__metaprograming.ipynb")

#### Building an advanced macro

In [61]:
macro assert(ex)
    return :( $ex ? nothing : throw(AssertionError($(string(ex)))))
end

@assert (macro with 1 method)

In [62]:
@assert 1 == 1.0

In [63]:
@assert 1 == 0

AssertionError: AssertionError: 1 == 0

In [64]:
macro assert(ex, msgs...)
    msg_body = isempty(msgs) ? ex : msgs[1]
    a = :($ex).args
    msg = "$(a[2])" * string(msg_body) * "$(a[3])"
    return :( $ex ? nothing : throw(AssertionError($msg)))
end

@assert (macro with 2 methods)

In [65]:
@macroexpand @assert a == b "a should equal b!"

:(if Main.a == Main.b
      Main.nothing
  else
      Main.throw(Main.AssertionError("aa should equal b!b"))
  end)

In [66]:
Expr(1==2)

TypeError: TypeError: in Expr, expected Symbol, got a value of type Bool

In [67]:
@assert 1==2 " is not equal to "

AssertionError: AssertionError: 1 is not equal to 2

#### Hygiene
- To avoid confilict, the default macro not change the variable in environment, use `esc()` we could change the variable.

In [68]:
macro time(ex)
    return quote
        local t0 = time_ns()
        local val = $(esc(ex))
        local t1 = time_ns()
        println("elapesed time: ", (t1-t0)/1e9, " seconds")
        val
    end
end



@time (macro with 1 method)

In [69]:
a = @time y = sin.(rand(1000));

elapesed time: 0.071787166 seconds


In [70]:
@time time_ns()

elapesed time: 2.09e-7 seconds


0x000098d03183bd57

In [71]:
macro zerox()
    return esc(:(x=0))
end
macro zerox_without_esc()
    return :(x=0)
end

@zerox_without_esc (macro with 1 method)

In [72]:
function foo()
    x = 1
    @zerox
    return x
end
function foo_without_esc()
    x = 1
    @zerox_without_esc
    return x
end

foo_without_esc (generic function with 1 method)

In [73]:
foo()

0

In [74]:
foo_without_esc()

1

In [75]:
macro time(expr)
    return :(timeit(() -> $(esc(expr))))
end

function timeit(f)
    t0 = time_ns()
    val = f()
    t1 = time_ns()
    println("elapsed time: ", (t1-t0)/1e9, " seconds")
    return val
end

timeit (generic function with 1 method)

In [76]:
@time sin.(rand(10000));

elapsed time: 7.4417e-5 seconds


#### Macros and dispatch

In [79]:
macro m end

macro m(args...)
    print("$(length(args)) arguments")
end

macro m(x,y)
    println("Two arguments")
end

@m (macro with 2 methods)

In [80]:
@m "asd"

1 arguments

In [81]:
@m "1" 2

Two arguments


- **Macro dispatch is based on the types of AST that are handed to the macro, not the types that the AST evaluates to at runtime.**

In [82]:
macro m(::Int)
    println("An Integer")
end

@m (macro with 3 methods)

In [83]:
@m 1

An Integer


In [88]:
x = 1
@m x

1 arguments

### Code Generation

In [89]:
struct MyNumber
    x::Float64
end

In [90]:
for op = (:sin, :cos, :tan, :log, :exp)
    eval(quote
        Base.$op(a::MyNumber) = MyNumber($op(a.x))
    end)
end

In [100]:
f(x::Real) = sin(x)
g(x::Real) = cos(x)

g (generic function with 1 method)

In [110]:
for op in (:f,:g)
    @eval $op(a::MyNumber) = MyNumber($op(a.x))
end

In [112]:
f1(x::Real) = sin(x);
g1(x::Real) = cos(x);

In [119]:
@eval begin
    f1(a::MyNumber) = MyNumber($f1(a.x))
    g1(a::MyNumber) = MyNumber($g1(a.x))
end

g1 (generic function with 2 methods)

In [91]:
x = MyNumber(pi)

MyNumber(3.141592653589793)

In [93]:
sin(x)

MyNumber(1.2246467991473532e-16)

In [94]:
cos(x)

MyNumber(-1.0)

### Generated functions
Generated functions have the capability to generate specialized code depending on the types of their arguments with more flexibility and/or less code than what can be ahieved with multiple dispatch.
1. You annotate the function declaration with the `@generated` macro. This adds some information to the AST that lets the compiler know that this is a generated function.
2. In the body of the generated function you **only have access to the types of the arguments** – not their values.
3. Instead of calculating something or performing some action, you **return a quoted expression** which, when evaluated, does what you want.
4. Generated functions are only permitted to call functions that were defined before the definition of the generated function. (Failure to follow this may result in getting MethodErrors referring to functions from a future world-age.)
5. Generated functions must not mutate or observe any non-constant global state (including, for example, IO, locks, non-local dictionaries, or using hasmethod). This means they can only read global constants, and cannot have any side effects. In other words, they must be completely pure. Due to an implementation limitation, this also means that they currently cannot define a closure or generator.
#### Using generated functions
- `@generated function`

In [137]:
@generated function foo(x)
    Core.println(x)
    return :(x * x)
end

foo (generic function with 2 methods)

In [138]:
x = foo(2);

Int64


In [139]:
y = foo("bar");

String


In [140]:
foo(1)

1

In [141]:
y

"barbar"